<a href="https://colab.research.google.com/github/obeabi/ProjectPortfolio/blob/master/TA_Weekly_Entry_etf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import yfinance as yf
import pandas as pd
import numpy as np
print("Libraries Installed!")

Libraries Installed!


In [16]:
# List of ETFs to analyze
df = pd.read_csv('etf_list.csv')
etfs = df['ETF'].to_list()

print(etfs)

['CQQQ', 'PGJ', 'GXC', 'FEZ', 'VGK', 'SPEU', 'XLC', 'EFAX', 'UNG', 'EWU', 'VOX', 'MORT', 'PWB', 'SPDW', 'VEA', 'EEMX', 'ARTY', 'VDC', 'PSCH', 'GWX', 'VWO', 'SPEM', 'QQQ', 'GMF', 'RTH', 'XLP', 'MGK', 'CHAT', 'VPL', 'VOOG', 'SPGM', 'SPYG', 'EWJ', 'VT']


In [11]:
# Function to fetch historical weekly data
def get_weekly_data(ticker):
  try:
      df = yf.download(ticker, period="2y", interval="1wk")
      df['20_week_SMA'] = df['Close'].rolling(window=20).mean()
      df['50_week_SMA'] = df['Close'].rolling(window=50).mean()
      df['RSI'] = compute_rsi(df['Close'])
      df['ATR'] = compute_atr(df, 14)
      df['OBV'] = compute_obv(df)
      df['10_week_avg_volume'] = df['Volume'].rolling(window=10).mean()
      return df
  except Exception as e:
      print("There is an error getting weekly data", e)

# Function to compute RSI
def compute_rsi(series, period=14):
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def compute_rsi2(series, period=14):
    try:
        # Ensure input is a Pandas Series, not a DataFrame
        if isinstance(series, pd.DataFrame):
            series = series.squeeze()  # Convert single-column DataFrame to Series

        # Check if series is empty
        if series is None or series.empty:
            raise ValueError("Input series is empty or None.")

        # Compute price changes
        delta = series.diff()

        # Separate gains and losses
        up = np.where(delta > 0, delta, 0)
        down = np.where(delta < 0, -delta, 0)

        # Compute RMA (Exponential Moving Average with alpha = 1/period)
        up_rma = pd.Series(up, index=series.index).ewm(span=period, adjust=False).mean()
        down_rma = pd.Series(down, index=series.index).ewm(span=period, adjust=False).mean()

        # Compute RSI
        rs = up_rma / down_rma
        rsi = 100 - (100 / (1 + rs))

        # Handle division by zero cases (same as TradingView)
        rsi[down_rma == 0] = 100
        rsi[up_rma == 0] = 0

        return rsi

    except Exception as e:
        print("Something went wrong while computing the RSI:", e)
        return None

# Function to compute ATR (Average True Range)
def compute_atr(df, period=14):
  try:
    df['High-Low'] = df['High'] - df['Low']
    df['High-Close'] = abs(df['High'] - df['Close'].shift(1))
    df['Low-Close'] = abs(df['Low'] - df['Close'].shift(1))
    df['TR'] = df[['High-Low', 'High-Close', 'Low-Close']].max(axis=1)
    return df['TR'].rolling(window=period).mean()
  except Exception as e:
      print("Something went wrong whilecomputing the ATR", e)


# Function to compute On-Balance Volume (OBV)
def compute_obv(df):
  try:
    # Calculate daily price change: 1 if price is up, -1 if down, 0 if unchanged
    price_change = df['Close'].diff()

    # Use price change to decide whether to add or subtract volume
    obv = (price_change > 0).astype(int) * df['Volume']  # Volume when price goes up
    obv -= (price_change < 0).astype(int) * df['Volume']  # Volume when price goes down

    # We accumulate the OBV by taking the cumulative sum of the volume changes
    obv = obv.cumsum()

    return obv
  except Exception as e:
      print("Something went wrong while computing the OBV", e)


# Function to calculate risk-reward ratio
def calculate_risk_reward(df):
  try:
    if df.empty or len(df) < 20:  # Ensure there are enough data points
        return np.nan

    latest_price = df['Close'].iloc[-1].iloc[0]

    # Use the ATR for setting support level
    atr = df['ATR'].iloc[-1]  # Latest ATR value
    atr_multiple = 1.25  # You can adjust this multiplier based on your strategy

    # Calculate the support level using the ATR
    trailing = atr * atr_multiple
    support_level = latest_price - trailing
    resistance_level = latest_price + (2*trailing)
    risk = latest_price- support_level
    reward = resistance_level - latest_price

    # Ensure risk is greater than zero before division
    if risk > 0:
        risk_reward_ratio = reward / risk
        return risk_reward_ratio if risk_reward_ratio > 0 else np.nan , support_level, resistance_level, latest_price, trailing
    else:
        return np.nan,np.nan, np.nan, np.nan, np.nan
  except Exception as e:
      print("Something went wrong whilecomputing the reward-risk ratio", e)

# Function to fetch daily data
def get_daily_data(ticker):
    df = yf.download(ticker, period="1y", interval="1d")
    df['20_day_SMA'] = df['Close'].rolling(window=20).mean()
    df['20_day_EMA'] = df['Close'].ewm(span=20, adjust=False).mean()
    df["Distance_EMA"] = (df['Close']/ df['Close'].ewm(span=20, adjust=False).mean() ) - 1
    df['RSI'] = compute_rsi(df['Close'])
    df['ATR'] = compute_atr(df, 20)
    return df

# Function to check weekly trend
def is_weekly_trend_bullish(df):
    if df.empty:
        return False

    latest_price = df['Close'].iloc[-1].iloc[0]
    latest_sma = df['20_week_SMA'].iloc[-1]
    latest_rsi = df['RSI'].iloc[-1]

    return (latest_price > latest_sma) and (latest_rsi > 50)

# Function to check daily entry signal
def is_daily_entry_signal(df):
    if df.empty:
        return False

    latest_price = df['Close'].iloc[-1].iloc[0]
    prev_price = df['Close'].iloc[-2].iloc[0]
    latest_sma = df['20_day_SMA'].iloc[-1]
    latest_rsi = df['RSI'].iloc[-1]
    latest_distance_20ema = df['Distance_EMA'].iloc[-1]

    # Look for a breakout above 20-day SMA & RSI > 55
    return (latest_price > latest_sma) and (latest_rsi > 55) and (latest_rsi < 75)

# Check entry conditions
def check_entry_conditions(tickers):
    results = []
    for ticker in tickers:
      df = get_daily_data(ticker)
      latest_price = df['Close'].iloc[-1].iloc[0]
      prev_price = df['Close'].iloc[-2].iloc[0]
      latest_sma = df['20_day_SMA'].iloc[-1]
      latest_rsi = df['RSI'].iloc[-1]
      latest_distance_20ema = df['Distance_EMA'].iloc[-1]

      if (latest_price > latest_sma) and (latest_rsi< 75) :
        if (latest_distance_20ema <= 0.02):
          entry_signal= "Enter full position"
        elif latest_distance_20ema < 0.05:
          entry_signal = "Enter 50% now and rest later"
        else:
          entry_signal= "Wait for pullback"
      else:
        entry_signal = "RSI is overbought- Wait for pullback"
      results.append([ticker, entry_signal])
    # Convert results to DataFrame
    df_results = pd.DataFrame(results, columns=["ETF", "Entry_Signal"])
    return df_results

# Multi-timeframe strategy check returning a DataFrame
def check_mtf_entry(tickers):
    results = []

    for ticker in tickers:
        weekly_df = get_weekly_data(ticker)
        daily_df = get_daily_data(ticker)

        if is_weekly_trend_bullish(weekly_df):
            if is_daily_entry_signal(daily_df):
                entry_signal = "Entry Confirmed ✅"
            else:
                entry_signal = "No Strong Entry Yet ⏳"
        else:
            entry_signal = "Weekly Trend Not Bullish ❌"

        results.append([ticker, entry_signal])

    # Convert results to DataFrame
    df_results = pd.DataFrame(results, columns=["ETF", "Entry_Signal"])
    return df_results

In [12]:
# Apply TA filters and prioritize ETFs
results = []
for etf in etfs:
  df =get_weekly_data(etf)
  price = df['Close'].iloc[-1].iloc[0]
  above_20SMA = price > df['20_week_SMA'].iloc[-1]
  above_50SMA = price > df['50_week_SMA'].iloc[-1]
  rsi_ok = 40 <= df['RSI'].iloc[-1] <= 70 # Not overbought or oversold
  volume_ok = df['Volume'].iloc[-1] > df['10_week_avg_volume'].iloc[-1] # Institutional interest
  volume_ok = volume_ok.iloc[0]

  # Calculate the OBV Moving Average
  df['OBV_EMA'] = df['OBV'].ewm(span=10, adjust=False).mean()
  # OBV trending up if current OBV is above the 20-period EMA
  obv_trending_up = df['OBV'].iloc[-1] > df['OBV_EMA'].iloc[-1]
  # obv_trending_up = df['OBV'].iloc[-1] > df['OBV'].iloc[-5] # OBV increasing over last 5 weeks
  # OBV trending down if current OBV is below the 20-period EMA
  obv_trending_down = df['OBV'].iloc[-1] < df['OBV_EMA'].iloc[-1]

  trend_ok = above_20SMA and above_50SMA

  if trend_ok and rsi_ok and (volume_ok or obv_trending_up):
  #if trend_ok and rsi_ok and (volume_ok and obv_trending_up):
    results.append({"ETF": etf })


# Multi-time frame entry Check
df_results = pd.DataFrame(results).dropna()
etfs_to_check = df_results['ETF'].tolist()
df_signals = check_mtf_entry(etfs_to_check)

df_signals


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,ETF,Entry_Signal
0,GDX,Entry Confirmed ✅
1,GDXJ,Entry Confirmed ✅
2,GLDM,No Strong Entry Yet ⏳
3,OUNZ,No Strong Entry Yet ⏳
4,GLD,Entry Confirmed ✅
5,RTH,No Strong Entry Yet ⏳
6,ITA,Weekly Trend Not Bullish ❌
7,PSCH,No Strong Entry Yet ⏳
8,SPEU,Entry Confirmed ✅
9,VOT,Entry Confirmed ✅


## Generate buy list

In [13]:
df_final = df_signals[df_signals['Entry_Signal'] =="Entry Confirmed ✅"]
final_etfs_to_check = df_final['ETF'].tolist()

buy_list = check_entry_conditions(final_etfs_to_check)

buy_list

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,ETF,Entry_Signal
0,GDX,Enter 50% now and rest later
1,GDXJ,Enter 50% now and rest later
2,GLD,Enter 50% now and rest later
3,SPEU,Enter 50% now and rest later
4,VOT,Enter full position
5,EWU,Enter 50% now and rest later
6,EFAX,Enter 50% now and rest later
7,PGJ,Wait for pullback
8,SPDW,Enter 50% now and rest later
9,VGK,Enter 50% now and rest later


In [14]:
# Apply TA filters and prioritize ETFs
results = []
for etf in buy_list['ETF'].to_list():
  df =get_weekly_data(etf)
  price = df['Close'].iloc[-1].iloc[0]
  above_20SMA = price > df['20_week_SMA'].iloc[-1]


  if  above_20SMA :
    rr_ratio,support_level, resistance_level, latest_price,trail = calculate_risk_reward(df)
    results.append({"ETF": etf ,"Risk-Reward": rr_ratio, "Support": support_level, "Resistance": resistance_level, "Current Price": latest_price, "Trail Price": trail})


# Sort ETFs by highest risk-to-reward ratio
df_results = pd.DataFrame(results).dropna().sort_values(by="Risk-Reward", ascending=False).reset_index(drop = True)
df_results


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

,ETF,Risk-Reward,Support,Resistance,Current Price,Trail Price
0,GLD,2.0,256.933756,285.002513,266.290009,9.356252
1,VGK,2.0,67.661323,75.277346,70.199997,2.538674
2,MORT,2.0,10.999383,12.651234,11.550000,0.550617
3,GDX,2.0,38.290453,46.659100,41.080002,2.789549
4,GDXJ,2.0,46.908873,58.222255,50.680000,3.771127
5,VOT,2.0,261.537787,295.234393,272.769989,11.232202
6,EFAX,2.0,41.818436,46.185729,43.274200,1.455764
7,PGJ,2.0,28.458093,34.313813,30.410000,1.951907
8,SPDW,2.0,35.613011,39.083978,36.770000,1.156989
9,XLF,2.0,49.992757,55.414483,51.799999,1.807242
